# Comparison between different users

This notebook is used to compare and analyze different methods of determining the length of a shaplet using the peak method.

Assumptions
* Each walking shaplet has a single peak far greater than the rest.

TODO
* Improve method to accept a shaplet with n peaks

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt
import peakutils
import math

## Peak Detection
This peak detection algorithm calculates peaks based on a given timeseries array and a threshold.  Refer to the [documentation](http://pythonhosted.org/PeakUtils/).  This method uses the peak detection algorithm to find the average distance between peaks.  If no peaks are found, the method returns nan.  

**Accuracy of the peak detection algorithm is assumed but not verified.  We should replace this with our custom peak detection algorithm**

In [2]:
def estimate_period(df, threshold, axis='x'):
    indexes = peakutils.indexes(df[axis].values, thres=threshold, min_dist=10)
    indexes = pd.Series(indexes)
    #peaks = df.iloc[indexes]
    #plt.close()
    #plt.plot(df['x'])
    #plt.plot(peaks['x'], 'r+')
    if indexes.index.size != 0:
        diff = indexes.drop(0) - indexes.shift(1).drop(0)
        return diff.mean()
    return float('nan')
    

Users who have data on walking: ['/102', '/103', '/107', '/110', '/116', '/118', '/121', '/122', '/125', '/131', '/133', '/140', '/143', '/144', '/148', '/149', '/153', '/159', '/161', '/166', '/171', '/174', '/179', '/181', '/182', '/184', '/188', '/189', '/192']

**Make sure your data file is in the right location on your local machine.**


In [3]:
walking116 = pd.read_csv('../../data/116/13_treadmill_3mph_0%.csv', names=["tick", "timestamp",
                                             "activity", "x", "y",
                                             "z", "user"], index_col=False)
walking125 = pd.read_csv('../../data/125/13_treadmill_3mph_0%.csv', names=["tick", "timestamp",
                                             "activity", "x", "y",
                                             "z", "user"], index_col=False)

## Cleaning data
I trimmed the beggining and end of each timeseries due to bogus values.  Points were chosen visually.

In [4]:
walking116 = walking116.iloc[2800:21000]
walking125 = walking125.iloc[2500:20000]
walking116 = walking116.reset_index()
walking125 = walking125.reset_index()

## Single Peak Detection
Our parameters are the time series to search, the window size, and the threshold to examine.  To improve accuracy, we are using a sliding window to improve the accuracy the peak detection algorithm.  Rather than find peaks over the entire data set, we use a fixed window length and calculate the average peak distance as the window slides over the time series from beginning to end.  The function gives control over how far to slide the window at each calucation and how big the window should be.  The function returns a list of lists of average peaks from the upper half and lower half of the time series. 

Parameters 
* timeseries(pandas dataframe) - dataframe of activity data. index starts at 0 and increases by 1.
* threshold(float between [0.,1.]) - Normalized threshold. Only the peaks with amplitude higher than the threshold will be detected.
* window_size(int) - size of the sliding window, less than the length of the time series.  If greater, window_size will be set to the length of the time series.
* axis(str) - either 'x', 'y', or 'z', corresponding to the axis to check
* step(int) - how far to slide the window for each calculation of peak distance.  If step is greater than window_size, the window size will be used as the step value.

Returns
* dic : 'upper' is upper peak averages, 'lower' is lower peak averages

In [5]:
def find_single_peak_dist(timeseries, threshold=0.8, window_size=500, axis='x', step=100):
    if (window_size > timeseries.last_valid_index()):
        window_size = timeseries.last_valid_index()
    if (axis not in ('x', 'y', 'z')):
        raise ValueError('invalid axis value, use x, y, or z')
    if (step > window_size):
        step = window_size
    upper = []
    lower = []
    ts_flipped = timeseries * -1
    for i in np.arange(0, timeseries.index.size - window_size, step):
        upper.append(estimate_period(timeseries.iloc[i:i+window_size], threshold, axis=axis))
        lower.append(estimate_period(ts_flipped.iloc[i:i+window_size], threshold, axis=axis))
    return { 'upper': upper, 'lower': lower, 'threshold': threshold, 'window': window_size }

## Choosing the correct window size
We get better accuracy with a smaller window size but we cannot go too small, otherwise our peak detection algorithm does not register any peaks.  If the window size is n and our shaplet size is s, n cannot be less than s otherwise we would never catch any consecutive pair of peaks.  Realistically, we would like catch multiple peaks at a time and average their distances together but at a minimum to be gaurenteed to catch a single period, n needs to be at least 2s.  Since we do not know the length of the shaplet before hand, we will experiment by testing a number of window sizes to determine correct size.

We will use the default threshold value of 0.8.

We will use the smallest window size that is able to get a prediction for at least 95% of the windows.  We will use a fixed starting value at 100 and increment by 100 until we fufill the above condition.

Set the initializing variables

In [31]:
accuracy = .99
starting_window_size = 100
increment_window_size = 100

Set function to capture to optimal window size.

In [51]:
def is_accurate(peak_avgs, accuracy=.99):
    return (1 - (peak_avgs.isnull().sum()/len(peak_avgs.index)) > accuracy)
    
def find_window_size(time_series, upper=True, accuracy=.99, starting_window_size=100, step=100):
    window_size = starting_window_size
    avg = pd.Series(math.nan)
    if upper:
        side = 'upper'
    else:
        side = 'lower'
        
    while not is_accurate(avg, accuracy):
        #print(is_accurate(avg, accuracy))
        info = find_single_peak_dist(time_series, window_size=window_size)
        avg = pd.Series(info[side])
        #print(avg.isnull().sum()/len(avg.index))
        window_size += increment_window_size
    return { 'window_size':(window_size - increment_window_size), 'avg':avg }

Find upper peak window size

In [52]:
upper = find_window_size(walking125, accuracy=accuracy, starting_window_size=starting_window_size)

In [53]:
upper['window_size']

300

Find the lower peak window size

In [54]:
lower = find_window_size(walking125, upper=False, accuracy=accuracy, starting_window_size=starting_window_size)

In [55]:
lower['window_size']

200

Plot the averages

In [56]:
plt.close()
plt.plot(upper['avg'])

In [57]:
plt.close()
plt.plot(lower['avg'])

In [58]:
upper['avg'].median()

102.0

In [59]:
lower['avg'].median()

44.29166666666667

Plot the histograms of the averages to show the clustering

In [63]:
plt.close()
plt.hist(upper['avg'].dropna(), bins=15, color='green')
plt.xlabel('averages')
plt.ylabel('frequency')

In [64]:
plt.close()
plt.hist(lower['avg'].dropna(), bins=15, color='red')
plt.xlabel('averages')
plt.ylabel('frequency')

## Upper vs lower
How do we select?  Discuss at next meeting.

## Shaplet search

### Selecting a shaplet
For now we will randomly select a shaplet.